In [ ]:
import sys; sys.path.append("../")
import numpy as np
import shared_info
from algos.functions import packet_error_probability
from envs.graph import Graph
from envs.graph_helper import import_oil_graph
from agent.Attrition_Agent import A_AttritionGathering_Agent, Dec_AttritionGathering_Agent, Global_AttritionGathering_Agent, Greedy_AttritionGathering_Agent, Reset_AttritionGathering_Agent
from copy import deepcopy
import seaborn as sns
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'pdf.fonttype': 42})
import logging

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.set_theme(context='paper', style='white', palette='deep', font='Times New Roman', font_scale=2, color_codes=True, rc=None)

distance = [i for i in range(0, 21)]
wind = [2.5, 5, 7.5, 10]
df_err = pd.DataFrame()

for w in wind:
    df = pd.DataFrame(data={'distance': distance, 'err': [packet_error_probability(i, w) for i in distance], 'wind': [w for _ in distance]})
    df_err = pd.concat([df_err, df], ignore_index=True)

ax = sns.pointplot(data=df_err, x='distance', y='err', hue='wind', markers=['x', 'D', 'o', 's'], palette=['b', 'g', 'r', 'skyblue'], linewidth=4)
ax.set_xlabel("Distance (km)")
ax.set_ylabel("Transmission Error Rate")
ax.set_xticks([0, 5, 10, 15, 20])
ax.legend(title='Wind speed (m/s)', frameon=True)
# fig.savefig("../../Figure/trans_err.pdf", bbox_inches='tight')

In [ ]:
# Log file for printing output.
print_output = logging.getLogger('print_output')
print_output.setLevel(logging.INFO)
formatter = logging.Formatter('%(message)s')
file_handler = logging.FileHandler('../../Data/print_output.txt')
file_handler.setFormatter(formatter)
print_output.addHandler(file_handler)

In [ ]:
# System parameters.
xL = 0                                           # min of the x-axis.
xH = 200                                         # max of the x-axis.
yL = 0                                           # min of the y-axis.
yH = 100                                         # max of the y-axis.
obsMask=np.array([[1,1,1,1,1,1,1,1],
                [0,1,1,1,1,1,1,1],
                [0,0,0,1,1,1,1,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0]])
agents = [100, 70]                               # agent starting position.

# Fixed parameters.
reward_radius = 2.5                               # reward disk radius (km).
N_components = 10                                 # number of components to be communicated.
N_comms_every = 10                                # period of tree compression.
c_p = 0.4                                         # Exploration parameter
gamma = 0.7                                       # discounting factor for d-ucb.

# Shared variational parameters.
budget = 200                                      # km.
planning_time = 60                                # (seconds).
n_agents = 20                                     # number of agents.
n_rewards = 200                                   # number of rewards.
attrition_intensity = 0.5                         # proportion of agents fail.

# A-MCTS.
msg_threshold = 1

# Initialise a seed for reproductivity.
rng =  np.random.default_rng(12345)

In [ ]:
# Parse the motion graph files.
G = Graph(xL, xH, yL, yH, reward_radius, obsMask)
G.add_node(agents[0], agents[1])
G, locs = import_oil_graph(G, n_rewards)

# Add reward.
rewards = rng.choice(locs, size=n_rewards, replace=False)
G.reset_reward(n_rewards)
G.add_reward(rewards)

In [ ]:
# Plot environment.
mpl.style.use('seaborn-v0_8-white')
fig, ax = plt.subplots(figsize=(20, 10))
img = plt.imread("../../Data/config/map.jpg")
ax.imshow(img, extent=[0, 200, 0, 100])
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.xlim(0, 200)
plt.ylim(0, 100)
# G.draw_edges()
# G.draw_nodes()
G.draw_agents([agents])
# G.draw_rewards(rewards)
for loc in locs:
    if loc in rewards:
        circle = plt.Circle((loc[0], loc[1]), 2.5, fc='none', ec='r', lw=1.5)
        plt.gca().add_patch(circle)
    plt.scatter(loc[0], loc[1], s=10, fc='gray')

In [ ]:
# Algorithm under testing
mode = 'A'

# Generate list of agents that will fail and the timestamp.
attrition_idx = rng.choice(range(n_agents), size=int(n_agents*attrition_intensity), replace=False)
attrition_time = {key: None for key in range(n_agents)}
for item in attrition_idx:
    attrition_time[item] = rng.random()*budget

# Create robots.
initial_actions = []
for j in range(len(G.edges_list)):
    if G.edges_list[j][0] == 0:
        initial_actions.append(j)

robots = list()
for i in range(n_agents):
    if mode == "Dec":
        robots.append(Dec_AttritionGathering_Agent(initial_state=np.nan, initial_actions=deepcopy(initial_actions), initial_position=deepcopy(agents), i=i, n_agents=n_agents, gamma=gamma, c_p=c_p, budget=budget, planning_time=planning_time, N_components=N_components, N_com_every=N_comms_every, Z=deepcopy(G), n_rewards=n_rewards, logger=print_output, fail_conidtion=attrition_time[i]))
    elif mode == "A":
        robots.append(A_AttritionGathering_Agent(initial_state=np.nan, initial_actions=deepcopy(initial_actions), initial_position=deepcopy(agents), i=i, n_agents=n_agents, gamma=gamma, c_p=c_p, budget=budget, planning_time=planning_time, N_components=N_components, N_com_every=N_comms_every, Z=deepcopy(G), n_rewards=n_rewards, RM_iter=100, msg_threshold=msg_threshold, logger=print_output, fail_conidtion=attrition_time[i]))
    elif mode == "Global":
        robots.append(Global_AttritionGathering_Agent(initial_state=np.nan, initial_actions=deepcopy(initial_actions), initial_position=deepcopy(agents), i=i, n_agents=n_agents, gamma=gamma, c_p=c_p, budget=budget, planning_time=planning_time, N_components=N_components, N_com_every=N_comms_every, Z=deepcopy(G), n_rewards=n_rewards, logger=print_output, fail_conidtion=attrition_time[i]))
    elif mode == "Greedy":
        robots.append(Greedy_AttritionGathering_Agent(initial_state=np.nan, initial_actions=deepcopy(initial_actions), initial_position=deepcopy(agents), i=i, n_agents=n_agents, gamma=gamma, c_p=c_p, budget=budget, planning_time=planning_time, N_components=N_components, N_com_every=N_comms_every, Z=deepcopy(G), n_rewards=n_rewards, RM_iter=100, msg_threshold=msg_threshold, logger=print_output, fail_conidtion=attrition_time[i]))
    elif mode == "Reset":
        robots.append(Reset_AttritionGathering_Agent(initial_state=np.nan, initial_actions=deepcopy(initial_actions), initial_position=deepcopy(agents), i=i, n_agents=n_agents, gamma=gamma, c_p=c_p, budget=budget, planning_time=planning_time, N_components=N_components, N_com_every=N_comms_every, Z=deepcopy(G), n_rewards=n_rewards, logger=print_output, fail_conidtion=attrition_time[i]))

# Set up shared information.
shared_info.init()
for robot in robots:
    shared_info.global_pos[robot.index] = deepcopy(robot.position)
    shared_info.global_distribution[robot.index] = deepcopy(robot.distribution)
    shared_info.joint_path[robot.index] = list()

In [ ]:
# Parallel planning and execution.
for robot in robots:
    robot.start()
for robot in robots:
    robot.join()

In [ ]:
final_paths = dict()
for idx in shared_info.joint_path.keys():
    if idx not in attrition_idx:
        final_paths[idx] = shared_info.joint_path[idx]
        print(idx, shared_info.joint_path[idx], G.evaluate_traj_cost(shared_info.joint_path[idx]))
print(sum(G.evaluate_traj_reward(final_paths))/n_rewards)

In [ ]:
f, axes = plt.subplots(figsize=(10, 6))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2, color_codes=True, rc=None)
path = dict()
score = np.array([])

max_iter = 0
for robot in robots:
    max_iter = max(max_iter, robot.iter)

for i in range(max_iter):
    for j in range(n_agents):
        try:
            path[j] = robots[j].rollout_history[i]
        except:
            pass
    score = np.append(score, sum(G.evaluate_traj_reward(path))/len(rewards))
ax = sns.lineplot(score)
ax.set_xlim([0, max_iter])

In [ ]:
mpl.style.use('seaborn-v0_8-white')
fig, ax = plt.subplots(figsize=(20, 10))
img = plt.imread("../../Data/config/map.jpg")
ax.imshow(img, extent=[0, 200, 0, 100])
# Turn off tick labels
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.xlim(0, 200)
plt.ylim(0, 100)

current_pos = list()

G.draw_agents([agents])

# final_paths = dict()
# final_paths[0] = [1, 1420, 16400]
# final_paths[1] = [5, 5988, 3177]
# final_paths[2] = [10, 13787, 11740, 17495]
# final_paths[3] = [15, 22645, 4720, 6590]
# final_paths[4] = [20, 34060, 7799]

for key in final_paths.keys():
    for i in range(len(final_paths[key])):
        x_index = []
        y_index = []
        if ~np.isnan(final_paths[key][i]):
            start_node = G.edges_list[int(final_paths[key][i])][0]
            end_node = G.edges_list[int(final_paths[key][i])][1]
            start_node_coor = G.find_coordinates(start_node)
            end_node_coor = G.find_coordinates(end_node)
            x_index.extend([start_node_coor[0], end_node_coor[0]])
            y_index.extend([start_node_coor[1], end_node_coor[1]])
            plt.plot(x_index, y_index, 'b', linewidth=3, solid_joinstyle='round', solid_capstyle='round')
    dx = (end_node_coor[0] - start_node_coor[0])/100
    dy = (end_node_coor[1] - start_node_coor[1])/100
    plt.arrow(end_node_coor[0], end_node_coor[1], dx, dy, shape='full', lw=2, length_includes_head=True, head_width=2, color='b')

for loc in locs:
    if loc in rewards:
        circle = plt.Circle((loc[0], loc[1]), 2.5, fc='none', ec='r', lw=1.5)
        plt.gca().add_patch(circle)
    plt.scatter(loc[0], loc[1], s=10, fc='gray')

plt.draw()
# fig.savefig("../../Figure/Oil_rigs.pdf")

In [ ]:
# # Generate graph for oil rigs inspection.
# from envs.prm import Euclindean_dist, SampleFree
# import csv

# def Near_radius(G, v, r):
#     U = []
#     for u in G.nodes.items():
#         dist = Euclindean_dist(u[1], v)
#         if dist >= 2 and dist <= r and (u[1][0] != v[0] or u[1][1] != v[1]):
#             U.append(u)
#     return np.array(U, dtype=object)

# G = Graph(xL=0, xH=20, yL=0, yH=10, obsMask=np.array([[1,1,1,1,1,1,1,1,1],
#                                                     [0,0,1,1,1,1,1,1,1],
#                                                     [0,0,0,0,1,1,1,1,0],
#                                                     [0,0,0,0,0,0,0,0,0],
#                                                     [0,0,0,0,0,0,0,0,0],
#                                                     [0,0,0,0,0,0,0,0,0],
#                                                     [0,0,0,0,0,0,0,0,0]]), radius=0.25)

# n_rewards = 1000

# agents = [10, 7]
# G.add_node(agents[0], agents[1])

# locs = np.genfromtxt("../../Data/config/locs.csv", delimiter=",")
# rng = np.random.default_rng()
# rewards = rng.choice(locs, size=n_rewards, replace=False)

# n_nodes = 1000
# nodes = SampleFree(n_nodes,
#                     G.xL,
#                     G.xH,
#                     G.yL,
#                     G.yH,
#                     G, seed=np.random.randint(100))

# for i in range(n_nodes):
#     G.add_node(nodes[i][0], nodes[i][1])

# for v in G.nodes.items():
#     U = Near_radius(G, v[1], r=2.5)
#     for u in U:
#         G.add_edge(v[0], u[0], Euclindean_dist(u[1], v[1]), n_rewards)

In [ ]:
# from agent.Central_Agent import Central_InfoGathering_Agent

# score = []
# for i in range(20):
#     # Add reward.
#     rewards = rng.choice(locs, size=n_rewards, replace=False)
#     G.reset_reward(n_rewards)
#     G.add_reward(rewards)
#     # Generate list of agents that will fail and the timestamp.
#     attrition_idx = rng.choice(range(n_agents), size=int(n_agents*0.5), replace=False)

#     # Create robots.
#     initial_actions = []
#     for j in range(len(G.edges_list)):
#         if G.edges_list[j][0] == 0:
#             initial_actions.append(j)

#     actions_to_try = dict()
#     for i in range(n_agents):
#         actions_to_try[i] = deepcopy(initial_actions)

#     robot = Central_InfoGathering_Agent(initial_state=np.nan, actions_to_try=actions_to_try, n_agents=n_agents, c_p=c_p, budget=200, planning_time=planning_time, Z=deepcopy(G), n_rewards=n_rewards)

#     robot.planning()

#     final_paths = dict()

#     for i in range(n_agents):
#         if i not in attrition_idx:
#             final_paths[i] = robot.tree.data.at[0, 'best_rollout_path'][i]
#     score.append(sum(G.evaluate_traj_reward(final_paths))/n_rewards)

# for i in range(n_agents):
#     path = robot.tree.data.at[0, 'best_rollout_path'][i]
#     print(G.evaluate_traj_cost(path))

# np.mean(score)

In [ ]:
# from envs.graph_helper import import_graph, parse
# import csv
# dir = "../../../INFOCOM-24/Data"
# N_config = 10
# N_trial = 2
# algos = ['Dec', 'Central', 'Global', 'Greedy', 'Reset', 'RM']

# df_cost = pd.DataFrame()

# for config in range(N_config):
#     G = Graph(-2, 2, -2, 2, 0.05)
#     G, agents, rewards, nodes, _, n_nodes, _  = import_graph(G, "{}/Config_{}".format(dir, config))
#     for trial in range(N_trial):
#         for algo in algos:
#             try:
#                 cost = list()
#                 with open("{}/9_action/action=9/{}-rollout-C{}-T{}.csv".format(dir, algo, config, trial+1), 'r') as csv_file:
#                     csv_reader = list(csv.reader(csv_file))
#                     joint_path = parse(csv_reader[-1][0])

#                 for path in joint_path.values():
#                     cost.append(G.evaluate_traj_cost(path))
#                 value = pd.DataFrame(data={'cost': cost, 'algo': [algo for _ in range(len(cost))]})
#                 df_cost = pd.concat([df_cost, value], ignore_index=True)
#             except:
#                 pass
# print(df_cost['cost'].mean(), df_cost['cost'].median())
# sns.boxplot(data=df_cost, x='algo', y='cost')